## Latent visualization space
Script for visualizing latent space for the RNNVAE. Visualization of selected dimensions, over time, testing visualization, colors, the like.

WARNING: messy Script

In [ ]:
# Imports
import sys
import os
sys.path.insert(0, os.path.abspath('./'))
import math
import torch
import torch.nn as nn
import numpy as np
import torch.utils
import torch.utils.data
from torchvision import datasets, transforms
from torch.autograd import Variable
from rnnvae.rnnvae import ModelRNNVAE
from sklearn.metrics import mean_absolute_error
from rnnvae.utils import open_MRI_data_var
from rnnvae.plot import plot_losses, plot_trajectory, plot_total_loss, pl

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Parameters, and load existing model
#hyperparameters
x_size = 40
h_size = 20
z_dim = 5
hidden = 5
n_layers = 1
n_epochs = 100
clip = 10
learning_rate = 1e-3
batch_size = 128
seed = 1714

# out_dir
out_dir = "experiments/base_test/"
if not os.path.exists(out_dir):
    os.makedirs(out_dir)

model = ModelRNNVAE(x_size, h_size, hidden, n_layers, 
                     hidden, n_layers, hidden,
                     n_layers, z_dim, hidden, n_layers,
                     clip, n_epochs, batch_size)

model.load(out_dir, 'model.pt')

In [ ]:
#Load data and predict
# LOAD DATA
#TODO: here we should load, someway, the labels corresponding to the subjects
# add a flag to the function to return them or something
#I would like to get, at least, age and dx
csv_path = "data/tadpole_mrionly.csv"
X_train, X_test = open_MRI_data_var(csv_path, train_set=0.8, normalize=True)

nfeatures = X_train[0].shape[1]

# Apply padding to both X_train and X_val
X_train_tensor = [ torch.FloatTensor(t) for t in X_train ]
X_train_pad = nn.utils.rnn.pad_sequence(X_train_tensor, batch_first=False)
X_test_tensor = [ torch.FloatTensor(t) for t in X_test ]
X_test_pad = nn.utils.rnn.pad_sequence(X_test_tensor, batch_first=False)

max_timepoints = X_train_pad.shape[0]
# Those datasets are of size [Tmax, Batch_size, nfeatures]

# Save mask to unpad later when testing
mask_train = X_train_pad > 0
mask_test = X_test_pad > 0

# Predict the reconstructions from X_val and X_train
X_test_fwd = model.predict(X_test_pad)
X_train_fwd = model.predict(X_train_pad)

Now start testing on the actual latent space. I want to program three main ways of latent space visualization:

1. Classic latent space visualization: selecting a specific time point and two specific latent dimensions, plot a 2D projection. Color by diagnosis? or by what?

2. Trajectory: for a specific latent dimension z_t, plot a 2d line over all timepoints. Plot all the subjects as lines, then try to visualize a general trajectory for that dimension? Maybe make the function flexible enough to 

3. Trajectory on 2D space: for two specific dimensions, plot a 2d space with all the timepoints z, and color the z by timepoints. See if there is a structure in the time over the selected dimensions of the latent space.

In [1]:
#First get the latent space
#Note that we have to apply the mask all the same!

z_train = X_train_fwd['zx']
z_test = X_test_fwd['zx']

z_train = np.array([Z[mask_train[:,i,:]].reshape((-1, nfeatures)).detach().numpy() for (i, Z) in enumerate(z_train)])
z_test = np.array([Z[mask_train[:,i,:]].reshape((-1, nfeatures)).detach().numpy() for (i, Z) in enumerate(z_train)])


In [ ]:
##1 Classic latent space visualization
dim0 = 0
dim1 = 1
z_t = 6

#prepare the data
train_z_d0 = np.array([x[z_t, dim0] if x.shape[0] > z_t else continue for x in z_train])
train_z_d1 = np.array([x[z_t, dim1] if x.shape[0] > z_t else continue for x in z_train])

test_z_d0 = np.array([x[z_t, dim0] if x.shape[0] > z_t else continue for x in z_test])
test_z_d1 = np.array([x[z_t, dim1] if x.shape[0] > z_t else continue for x in z_test])

#plot latent space
#plot both train and test together?
plt.figure()
sns.scatterplot(x=train_z_d0, y=train_z_d1)
sns.scatterplot(x=test_z_d0, y=test_z_d1)
plt.show()
#plt.savefig(f'{out_dir}/latent_space.png')
#plt.close()




In [ ]:
# 2 Trajectory latent space visualization
dim = 0

#prepare the data
train_z_d0 = [x[:, dim0] if x.shape[0] > z_t else continue for x in z_train]
test_z_d0 = [x[:, dim0] if x.shape[0] > z_t else continue for x in z_test]

plt.figure()

#plot all lines
#regarding the x axis: maybe it make sense to not use timepoints but...
#age?
# time from baseline?
#age probably is best
for line in train_z_d0:
    plt.plot(list(range(len(line))), line)

#compute an approximation
#interpolate for the cases where we do not have enough points
#from  https://stackoverflow.com/questions/51933785/getting-a-mean-curve-of-several-curves-with-x-values-not-being-the-same?noredirect=1&lq=1

mean_x_axis = list(range(max_timepoints))
ys_interp = [np.interp(mean_x_axis, list(range(len(x[:, i]))), x[:, i]) for x in train_z_d0]
mean_y_axis = np.mean(ys_interp, axis=0)

#plot it too
plt.plot(mean_x_axis, mean_y_axis)
plt.show()

In [ ]:
#3 2D Trajectory latent space visualization
dim0=0
dim1=1

plt.figure()

# create color cmap
pallete = sns.color_palette("viridis", max_timepoints)

for tp in range(max_timepoints):

    train_z_d0 = [x[tp, dim0] if x.shape[0] > z_t else continue for x in z_train]
    test_z_d0 = [x[tp, dim0] if x.shape[0] > z_t else continue for x in z_test]

    train_z_d1 = [x[tp, dim1] if x.shape[0] > z_t else continue for x in z_train]
    test_z_d1 = [x[tp, dim1] if x.shape[0] > z_t else continue for x in z_test]

    #but color by timepoint
    sns.scatterplot(x=train_z_d0, y=train_z_d1, color=pallete[tp])
    sns.scatterplot(x=test_z_d0, y=test_z_d1, color=pallete[tp])

plt.show()

